In [ ]:
import yfinance as yf
import mysql.connector
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import datetime
import numpy as np



In [ ]:
# Configuration
STOCK_SYMBOL = "AAPL"
DB_HOST = "desktop-gcq0gr6"
DB_USER = "root"
DB_PASSWORD = "Ansh@1234"
DB_NAME = "hehe"
TABLE_NAME = f"{STOCK_SYMBOL.lower()}_daily_prices"


In [ ]:
def fetch_stock_data(symbol, period="1y"):
    try:
        data = yf.download(symbol, period=period)
        if data is not None and not data.empty:
            return data
        else:
            print(f"Error: Could not retrieve data for {symbol} from yfinance.")
            return None
    except Exception as e:
        print(f"Error fetching data from yfinance: {e}")
        return None

def connect_db():
    try:
        mydb = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        return mydb
    except mysql.connector.Error as err:
        print(f"Error connecting to MySQL: {err}")
        return None

def create_table(mydb, table_name):
    mycursor = mydb.cursor()
    try:
        mycursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            date DATE PRIMARY KEY,
            open_price DECIMAL(10, 2),
            high_price DECIMAL(10, 2),
            low_price DECIMAL(10, 2),
            close_price DECIMAL(10, 2),
            adjusted_close DECIMAL(10, 2),
            volume BIGINT,
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)
        mydb.commit()
        print(f"Table '{table_name}' created or already exists.")
    except mysql.connector.Error as err:
        print(f"Error creating table '{table_name}': {err}")

def store_data_in_db(mydb, table_name, df):
    mycursor = mydb.cursor()
    insert_query = f"""
    INSERT INTO {table_name} (date, open_price, high_price, low_price, close_price, adjusted_close, volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        open_price = VALUES(open_price),
        high_price = VALUES(high_price),
        low_price = VALUES(low_price),
        close_price = VALUES(close_price),
        adjusted_close = VALUES(adjusted_close),
        volume = VALUES(volume),
        timestamp = CURRENT_TIMESTAMP
    """
    try:
        for index, row in df.iterrows():
            values = (
                index.strftime('%Y-%m-%d'),
                float(row.Open),
                float(row.High),
                float(row.Low),
                float(row.Close),
                float(row['Adj Close']),  # Correct adjusted close here
                int(row.Volume)
            )
            mycursor.execute(insert_query, values)
        mydb.commit()
        print(f"Data successfully stored in table '{table_name}'.")
    except KeyError as e:
        print(f"Error accessing column in DataFrame: {e}")
        mydb.rollback()
    except mysql.connector.Error as err:
        print(f"Error inserting data into table '{table_name}': {err}")
        mydb.rollback()



In [ ]:
def fetch_data_for_training(mydb, table_name):
    mycursor = mydb.cursor()
    query = f"SELECT date, adjusted_close, volume FROM {table_name} ORDER BY date ASC"
    mycursor.execute(query)
    results = mycursor.fetchall()
    dates = [res[0] for res in results]
    prices = [float(res[1]) for res in results]
    volumes = [int(res[2]) for res in results]
    return pd.DataFrame({'date': dates, 'adjusted_close': prices, 'volume': volumes})

def add_features(df):
    # Lagged features for close price and volume (1 to 3 days)
    for lag in range(1, 4):
        df[f'close_price_lag{lag}'] = df['adjusted_close'].shift(lag)
        df[f'volume_lag{lag}'] = df['volume'].shift(lag)

    # Simple Moving Average (SMA) 5-day window
    df['SMA_5'] = df['adjusted_close'].rolling(window=5).mean()

    # Exponential Moving Average (EMA) 5-day window
    df['EMA_5'] = df['adjusted_close'].ewm(span=5, adjust=False).mean()

    # Relative Strength Index (RSI) with 14-day window
    delta = df['adjusted_close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI_14'] = 100 - (100 / (1 + rs))

    df.dropna(inplace=True)  # Drop rows with NaNs created by rolling calculations
    return df


In [ ]:
def train_model(df):
    if df is None or df.empty:
        print("Error: No data available for training.")
        return None, None, None, None, None

    df = add_features(df)

    # Select features for training
    feature_cols = [col for col in df.columns if col not in ['date', 'adjusted_close', 'volume']]
    X = df[feature_cols]
    y = df['adjusted_close']

    # Scaling features and target
    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()

    X_scaled = x_scaler.fit_transform(X)
    y_scaled = y_scaler.fit_transform(y.values.reshape(-1, 1))

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)

    return model, x_scaler, y_scaler, X_test, y_test


def evaluate_model(model, x_test, y_test, y_scaler):
    """Evaluate model performance on test data and print metrics."""
    y_pred_scaled = model.predict(x_test)
    # Inverse transform to original scale
    y_test_inv = y_scaler.inverse_transform(y_test)
    y_pred_inv = y_scaler.inverse_transform(y_pred_scaled)

    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    mse = mean_squared_error(y_test_inv, y_pred_inv)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_inv, y_pred_inv)

    print(f"\n📊 Model Evaluation Metrics:")
    print(f"MAE  (Mean Absolute Error): {mae:.4f}")
    print(f"MSE  (Mean Squared Error): {mse:.4f}")
    print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")



In [ ]:
def predict_next_day_price(mydb, table_name, model, x_scaler, y_scaler):
    if model is None or x_scaler is None or y_scaler is None:
        print("Error: Model or scaler not available for prediction.")
        return None

    mycursor = mydb.cursor()
    # Fetch enough historical data to compute indicators (about last 30 days)
    query = f"SELECT date, adjusted_close, volume FROM {table_name} ORDER BY date DESC LIMIT 30"
    mycursor.execute(query)
    results = mycursor.fetchall()

    if len(results) < 15:  # RSI needs 14 days, so check minimum
        print("Error: Not enough historical data to generate features for prediction.")
        return None

    # Reverse to ascending date order for feature calculation
    results = results[::-1]

    # Create dataframe
    df = pd.DataFrame(results, columns=['date', 'adjusted_close', 'volume'])

    # Add features using your earlier function
    df = add_features(df)

    if df.empty:
        print("Error: Feature dataframe is empty after processing.")
        return None

    # The latest row contains all features except target (which we want to predict)
    feature_cols = [col for col in df.columns if col not in ['date', 'adjusted_close', 'volume']]
    latest_features = df.iloc[-1][feature_cols].values.reshape(1, -1)

    # Scale the features
    latest_scaled = x_scaler.transform(latest_features)

    # Predict
    predicted_price_scaled = model.predict(latest_scaled)

    # Inverse scale the predicted price
    predicted_price = y_scaler.inverse_transform(predicted_price_scaled.reshape(-1, 1))[0][0]

    next_day = datetime.date.today() + datetime.timedelta(days=1)
    print(f"Predicted closing price for {next_day}: {predicted_price:.2f}")
    return predicted_price


In [ ]:
if __name__ == "__main__":
    # Connect to the database
    mydb = connect_db()
    if mydb is None:
        exit()

    # Create table if it doesn't exist
    create_table(mydb, TABLE_NAME)

    # Fetch stock data and store in DB
    historical_data_df = fetch_stock_data(STOCK_SYMBOL, period="1y")
    if historical_data_df is not None:
        store_data_in_db(mydb, TABLE_NAME, historical_data_df)

        # Prepare training data
        training_data_df = fetch_data_for_training(mydb, TABLE_NAME)
        print("Head of training_data_df:")
        print(training_data_df.head())

        # Train model
        model, x_scaler, y_scaler, X_test, y_test = train_model(training_data_df.copy())

        # Evaluate model
        if model and X_test is not None and y_test is not None:
            evaluate_model(model, X_test, y_test, y_scaler)

        # Predict next day's price
        if model and x_scaler and y_scaler:
            predicted_price = predict_next_day_price(mydb, TABLE_NAME, model, x_scaler, y_scaler)

    # Close DB connection
    if mydb and mydb.is_connected():
        mydb.close()
        print("Database connection closed.")